# FakeNewsNetInference

This notebook helps users distinguish between fake and real news titles using CNN models created in the FakeNewsNetCNN notebook. The models were trained on the FakeNewsNet dataset, the GossipCop and PoliticFact subsets. For more information, please see the [project folder](https://drive.google.com/drive/folders/1kLJKLE8DZHQQf9OqaSHqlehi_5j4EX9n), start with the Report.pdf.

## Instructions for Use

- **Type the news title** into the text area below and select the type of news (Gossip or Political) from the dropdown menu.
- Press the "Check Fake News" button.
- The model will predict and display whether the news is likely **Fake** or **Not Fake** according to the selected dataset.
- To show or hide the code, click the arrow (>) on the left side of the "Inference code" cell.
- Please note: The models work best with English text.

## Handling Disconnections


- If the Colab runtime disconnects (noticeable at the top-right corner of the page), please reconnect by clicking the 'Connect' button.
- After reconnecting, re-run the cell containing the Inference code.

In [8]:
# @title # Inference code

# Install libraries with specified versions for compatibility of model loading and import necessary libraries
!pip install tensorflow==2.17.0 gdown==5.1.0 joblib==1.4.2 ipywidgets==7.7.1

import tensorflow as tf
import ipywidgets as widgets
from IPython.display import display, HTML
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib
import gdown
import zipfile
import os
import time
import re

# Public links to files
gossipcop_model_url = 'https://drive.google.com/uc?id=1tgL8ITlc61l0BnjDLbnyOJ8zghtNDjVL'
gossipcop_tokenizer_file_url = 'https://drive.google.com/uc?id=12e6MIz9qKCbLu6cz7zyRytOVAwKPOMYI'
gossipcop_label_encoder_file_url = 'https://drive.google.com/uc?id=1PT-7nFgXT0qISSRqEe9jTMJdAro5aQ3j'
politifact_model_url = 'https://drive.google.com/uc?id=14wU7pqr6tlpg3bE89lx3Y-Nf0O_Q4Mwy'
politifact_tokenizer_file_url = 'https://drive.google.com/uc?id=19nmqTyZoXZcbc24Gw7lYEKyLNVoKGine'
politifact_label_encoder_file_url = 'https://drive.google.com/uc?id=1wHBlUc4xLZYdbkW8yVmmVQtxpKyHAftD'

# Download files from Google Drive
gdown.download(gossipcop_model_url, "best_cnn_model_gossipcop.keras", quiet=False)
gdown.download(gossipcop_tokenizer_file_url, "tokenizer_gossipcop.pkl", quiet=False)
gdown.download(gossipcop_label_encoder_file_url, "label_encoder_gossipcop.pkl", quiet=False)
gdown.download(politifact_model_url, "best_cnn_model_politifact.keras", quiet=False)
gdown.download(politifact_tokenizer_file_url, "tokenizer_politifact.pkl", quiet=False)
gdown.download(politifact_label_encoder_file_url, "label_encoder_politifact.pkl", quiet=False)

# Load the CNN models
model_gossipcop = tf.keras.models.load_model('best_cnn_model_gossipcop.keras')
model_politifact = tf.keras.models.load_model('best_cnn_model_politifact.keras')

# Load tokenizers and label encoders
tokenizer_gossipcop = joblib.load('tokenizer_gossipcop.pkl')
le_gossipcop = joblib.load('label_encoder_gossipcop.pkl')
tokenizer_politifact = joblib.load('tokenizer_politifact.pkl')
le_politifact = joblib.load('label_encoder_politifact.pkl')

print("Setup complete!")

# Function to preprocess text
def preprocess_text(text, tokenizer):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=100)
    return padded

# Define widgets for user input
text_input = widgets.Textarea(
    value='',
    placeholder='Type the news title here...',
    description='News Title:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='200px')
)

news_type = widgets.Dropdown(
    options=['Gossip', 'Political'],
    value='Gossip',
    description='News Type:',
    disabled=False,
)

button = widgets.Button(description="Check Fake News", tooltip="Click to check if the news is fake or not")
clear_button = widgets.Button(description="Clear", tooltip="Clear the input text")
output = widgets.Output()
progress = widgets.IntProgress(value=0, min=0, max=100, step=10, description='Processing:', bar_style='info', style={'bar_color': '#00BFFF'})

# Function to validate input text
def validate_input(text):
    if not text.strip():
        return False, "Input is empty. Please enter a news title."
    if len(re.findall(r'\w+', text)) < 3:  # Check for minimum number of words
        return False, "Input is too short or not meaningful. Please enter a valid news title."
    return True, ""

# Precompile the prediction functions
gossipcop_predict_fn = model_gossipcop.make_predict_function()
politifact_predict_fn = model_politifact.make_predict_function()

# Button click event handler
def on_button_clicked(b):
    with output:
        output.clear_output()
        progress.value = 0
        input_text = text_input.value
        is_valid, message = validate_input(input_text)
        if not is_valid:
            display(HTML(f"<p style='color: red; font-size: 16px;'>{message}</p>"))
            return

        progress.value = 20
        display(progress)
        for i in range(8):
            time.sleep(0.1)
            progress.value += 10

        news_type_selected = news_type.value

        if news_type_selected == 'Gossip':
            # Preprocess the text
            preprocessed_text = preprocess_text(input_text, tokenizer_gossipcop)
            progress.value = 90
            # Making predictions
            prediction = model_gossipcop(preprocessed_text)
            progress.value = 100
            # Display predictions
            result = "<p style='color: green; font-size: 20px; background-color: #CCFFCC; padding: 10px; border-radius: 5px;'>GossipCop Prediction: Not Fake</p>" if prediction[0][0] <= 0.5 else "<p style='color: red; font-size: 20px; background-color: #FFCCCC; padding: 10px; border-radius: 5px;'>GossipCop Prediction: Fake</p>"

        else:
            # Preprocess the text
            preprocessed_text = preprocess_text(input_text, tokenizer_politifact)
            progress.value = 90
            # Making predictions
            prediction = model_politifact(preprocessed_text)
            progress.value = 100
            # Display predictions
            result = "<p style='color: green; font-size: 20px; background-color: #CCFFCC; padding: 10px; border-radius: 5px;'>PoliticFact Prediction: Not Fake</p>" if prediction[0][0] <= 0.5 else "<p style='color: red; font-size: 20px; background-color: #FFCCCC; padding: 10px; border-radius: 5px;'>PoliticFact Prediction: Fake</p>"

        display(HTML(result))
        progress.value = 0

button.on_click(on_button_clicked)

# Clear button click event handler
def on_clear_clicked(b):
    text_input.value = ''
    output.clear_output()

clear_button.on_click(on_clear_clicked)

# Display widgets
display(news_type, text_input, button, clear_button, output)

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


Downloading...
From: https://drive.google.com/uc?id=1tgL8ITlc61l0BnjDLbnyOJ8zghtNDjVL
To: /content/best_cnn_model_gossipcop.keras
100%|██████████| 2.68M/2.68M [00:00<00:00, 167MB/s]
Downloading...
From: https://drive.google.com/uc?id=12e6MIz9qKCbLu6cz7zyRytOVAwKPOMYI
To: /content/tokenizer_gossipcop.pkl
100%|██████████| 479k/479k [00:00<00:00, 50.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PT-7nFgXT0qISSRqEe9jTMJdAro5aQ3j
To: /content/label_encoder_gossipcop.pkl
100%|██████████| 343/343 [00:00<00:00, 913kB/s]
Downloading...
From: https://drive.google.com/uc?id=14wU7pqr6tlpg3bE89lx3Y-Nf0O_Q4Mwy
To: /content/best_cnn_model_politifact.keras
100%|██████████| 9.24M/9.24M [00:00<00:00, 228MB/s]
Downloading...
From: https://drive.google.com/uc?id=19nmqTyZoXZcbc24Gw7lYEKyLNVoKGine
To: /content/tokenizer_politifact.pkl
100%|██████████| 104k/104k [00:00<00:00, 39.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wHBlUc4xLZYdbkW8yVmmVQtxpKyHAftD
To: /content/label_encode

Setup complete!


Dropdown(description='News Type:', options=('Gossip', 'Political'), value='Gossip')

Textarea(value='', description='News Title:', layout=Layout(height='200px', width='100%'), placeholder='Type t…

Button(description='Check Fake News', style=ButtonStyle(), tooltip='Click to check if the news is fake or not'…

Button(description='Clear', style=ButtonStyle(), tooltip='Clear the input text')

Output()